In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_with_lag/data_with_lag.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_6"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



         Umsatz  Temp_Very_Cold_current  Temp_Very_Cold_prev  \
9042  41.813038                       0                  0.0   
9043  16.410876                       0                  0.0   
9044  18.574458                       0                  0.0   
9045  24.640084                       0                  0.0   
9046  43.515366                       0                  0.0   
...         ...                     ...                  ...   
9329  87.471228                       0                  0.0   
9330  71.911652                       0                  0.0   
9331  84.062223                       0                  0.0   
9332  60.981969                       0                  0.0   
9333  34.972644                       0                  0.0   

      Temp_Very_Cold_next  Temp_Cold_current  Temp_Cold_prev  Temp_Cold_next  \
9042                  0.0                  0             0.0             0.0   
9043                  0.0                  0             0.0           

In [2]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Zeilen- und Spaltenanzeige in Pandas erweitern
pd.set_option("display.max_rows", None)  # Zeigt alle Zeilen
pd.set_option("display.max_columns", None)  # Zeigt alle Spalten

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                            1.000000
Weihnachtsmarkt_current           0.654958
Weihnachtsmarkt_prev              0.621177
Weihnachtsmarkt_next              0.603408
Monat_12_next                     0.440972
Winter_next                       0.395443
Monat_12_current                  0.376204
Monat_12_prev                     0.339219
Winter_current                    0.329182
Winter_prev                       0.294377
Temp_Cold_prev                    0.238473
Temp_Cold_current                 0.219983
Temp_Cold_next                    0.217073
Cloud_Cloudy_prev                 0.176919
Cloud_Cloudy_next                 0.151832
Freitag_prev                      0.129857
Weather_Severe_next               0.117002
VPI_current                       0.112808
VPI_next                          0.110240
Werktag_prev                      0.109846
Samstag_current                   0.108345
VPI_prev                          0.097273
Wind_Moderate_next                0.091207
wetter_sehr

In [3]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)



Weihnachtsmarkt_current           0.294963
Weihnachtsmarkt_prev              0.253984
Weihnachtsmarkt_next              0.229332
Monat_12_next                     0.156260
Herbst_next                       0.136578
Winter_next                       0.136578
Monat_12_current                  0.131507
Herbst_current                    0.115644
Winter_current                    0.115644
Monat_12_prev                     0.114569
VPI_current                       0.113153
VPI_next                          0.109471
Winter_prev                       0.100911
Monat_11_next                     0.099872
Herbst_prev                       0.098915
Monat_11_current                  0.085559
VPI_prev                          0.077333
Monat_11_prev                     0.074921
Temp_Cold_prev                    0.054164
Wind_Moderate_next                0.052090
Cloud_Cloudy_prev                 0.042227
Cloud_ok_prev                     0.042227
Wind_Light_prev                   0.035925
Monat_10_cu

In [4]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Temp_Cold_current', 'Temp_Cold_prev', 'Temp_Cold_next', 'Temp_Mild_current', 'Temp_Mild_prev', 'Temp_Mild_next', 'Temp_Hot_prev', 'Cloud_Cloudy_current', 'Cloud_Cloudy_prev', 'Cloud_Cloudy_next', 'Wind_Light_current', 'Wind_Light_prev', 'Wind_Light_next', 'Wind_Moderate_current', 'Wind_Moderate_prev', 'Wind_Moderate_next', 'Wind_Strong_current', 'Wind_Strong_prev', 'Wind_Strong_next', 'Weather_Light_Issues_current', 'Weather_Light_Issues_prev', 'Weather_Light_Issues_next', 'Weather_Moderate_current', 'Weather_Moderate_prev', 'Weather_Moderate_next', 'Montag_current', 'Montag_prev', 'Montag_next', 'Dienstag_current', 'Dienstag_prev', 'Dienstag_next', 'Mittwoch_current', 'Mittwoch_prev', 'Mittwoch_next', 'Donnerstag_current', 'Donnerstag_prev', 'Donnerstag_next', 'Freitag_current', 'Freitag_prev', 'Freitag_next', 'Samstag_current', 'Samstag_prev', 'Samstag_next', 'Sonntag_current', 'Sonntag_prev', 'Sonntag_next', 'VPI_current', 'VPI_prev', 'VPI_next', 'Numbe

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                            Feature         VIF
0            Temp_Very_Cold_current         inf
1               Temp_Very_Cold_prev         inf
2               Temp_Very_Cold_next         inf
3                 Temp_Cold_current         inf
4                    Temp_Cold_prev         inf
5                    Temp_Cold_next         inf
6                 Temp_Mild_current         inf
7                    Temp_Mild_prev         inf
8                    Temp_Mild_next         inf
9                 Temp_Warm_current         NaN
10                   Temp_Warm_prev         NaN
11                   Temp_Warm_next         NaN
12                 Temp_Hot_current         NaN
13                    Temp_Hot_prev         inf
14                    Temp_Hot_next         NaN
15              Cloud_Clear_current         inf
16                 Cloud_Clear_prev         inf
17                 Cloud_Clear_next         inf
18      Cloud_Partly_Cloudy_current         inf
19         Cloud_Partly_Cloudy_prev     

In [6]:
# suche miteinander korrelierrende features

In [7]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Mittwoch_next', 'Dienstag_current'), ('Werktag_next', 'Sonntag_next'), ('Monat_12_current', 'Weihnachtsmarkt_current'), ('Monat_10_prev', 'Temp_Mild_next'), ('Cloud_ok_next', 'Cloud_Partly_Cloudy_next'), ('Monat_11_prev', 'Weihnachtsmarkt_prev'), ('Monat_12_next', 'Winter_prev'), ('Monat_12_next', 'Weihnachtsmarkt_next'), ('Winter_prev', 'Weihnachtsmarkt_prev'), ('Weather_Moderate_prev', 'Weather_Light_Issues_prev'), ('Winter_current', 'Herbst_current'), ('is_holiday_prev', 'is_holiday_current'), ('Heimspiel_prev', 'Sonntag_current'), ('Temp_Mild_prev', 'Temp_Cold_next'), ('Monat_11_current', 'Weihnachtsmarkt_next'), ('Monat_11_current', 'Winter_prev'), ('Monat_12_next', 'Winter_current'), ('Wind_ok_current', 'Wind_Strong_current'), ('Temp_Very_Cold_prev', 'Temp_Very_Cold_current'), ('Monat_11_next', 'Winter_next'), ('Wind_Moderate_next', 'Wind_Light_next'), ('Monat_11_current', 'Winter_current'), ('Monat_12_prev', 'Weihnachtsmarkt_prev'), ('Monat_12